<a href="https://colab.research.google.com/github/greeshmapj/AI-ML-nov2025-assignments/blob/main/Greeshma_Assignment5_ANN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#1. Import Libraries

In [1]:
!pip install keras-tuner

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.4/129.4 kB 4.0 MB/s eta 0:00:00


In [2]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

import tensorflow as tf
import keras_tuner as kt
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.utils import to_categorical

#2.Load Dataset

In [3]:
df = pd.read_csv('/content/mobile_price_classification.csv')
df.head()

,battery_power,bluetooth,clock_speed,dual_sim,front_cam,4G,int_memory,m_dep,mobile_wt,n_cores,...,px_height,px_width,ram,sc_h,sc_w,talk_time,three_g,touch_screen,wifi,price_range
0,842,0,2.2,0,1,0,7,0.6,188,2,...,20,756,2549,9,7,19,0,0,1,1
1,1021,1,0.5,1,0,1,53,0.7,136,3,...,905,1988,2631,17,3,7,1,1,0,2
2,563,1,0.5,1,2,1,41,0.9,145,5,...,1263,1716,2603,11,2,9,1,1,0,2
3,615,1,2.5,0,0,0,10,0.8,131,6,...,1216,1786,2769,16,8,11,1,0,0,2
4,1821,1,1.2,0,13,1,44,0.6,141,2,...,1208,1212,1411,8,2,15,1,1,0,1


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 21 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   battery_power   2000 non-null   int64  
 1   bluetooth       2000 non-null   int64  
 2   clock_speed     2000 non-null   float64
 3   dual_sim        2000 non-null   int64  
 4   front_cam       2000 non-null   int64  
 5   4G              2000 non-null   int64  
 6   int_memory      2000 non-null   int64  
 7   m_dep           2000 non-null   float64
 8   mobile_wt       2000 non-null   int64  
 9   n_cores         2000 non-null   int64  
 10  primary_camera  2000 non-null   int64  
 11  px_height       2000 non-null   int64  
 12  px_width        2000 non-null   int64  
 13  ram             2000 non-null   int64  
 14  sc_h            2000 non-null   int64  
 15  sc_w            2000 non-null   int64  
 16  talk_time       2000 non-null   int64  
 17  three_g         2000 non-null   i

**INSIGHT**
* No missing values
* 20 feature column present
* Has numerical columns only

#3. Separate features & Label

In [5]:
X = df.drop('price_range',axis=1)
y = df['price_range']

#4. Train-Test Split

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#5. One-Hot Encode Labels

In [7]:
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

#6. Standardize Features

In [8]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

#7. Build ANN Model Function

In [9]:
def build_model(hp):
    model = Sequential()

    # Tune number of neurons
    model.add(Dense(
        units=hp.Int('units_1', min_value=8, max_value=64, step=8), #8,16,24,32,40,48,56,64
        activation='relu',
        input_shape=(X_train.shape[1],)
    )) #hidden layer1

    # Tune second hidden layer neurons
    model.add(Dense(
        units=hp.Int('units_2', min_value=8, max_value=64, step=8),
        activation='relu'
    ))

    # Tune dropout
    model.add(Dropout(
        rate=hp.Choice('dropout_rate', values=[0.2, 0.3, 0.4])
    ))

    model.add(Dense(4, activation='softmax'))

    # Tune learning rate
    model.compile(
        optimizer=tf.keras.optimizers.Adam(
            learning_rate=hp.Choice('learning_rate', values=[0.01, 0.001, 0.0001])
        ),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    return model

#8. Initialize Keras Tuner

In [10]:
# 8️⃣ Initialize Tuner=1
tuner = kt.RandomSearch(
    build_model, # we built it earlier
    objective='val_accuracy',
    max_trials=10, #10 random combinations
    executions_per_trial=1,
    directory='mobile_price_tuning',
    project_name='mobile_price_ann'
)

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


#9. Search Best Hyperparameters

In [11]:
# 9️⃣ Search Best Hyperparameters
tuner.search(
    X_train, y_train,
    epochs=50,
    validation_split=0.2,
    batch_size=32,
    verbose=1
)

Trial 10 Complete [00h 00m 16s]
val_accuracy: 0.59375

Best val_accuracy So Far: 0.940625011920929
Total elapsed time: 00h 02m 56s


So 93.75% is performance on unseen validation data during tuning.

#10. Get Best Model

In [12]:
best_model = tuner.get_best_models(num_models=1)[0]

/usr/local/lib/python3.12/dist-packages/keras/src/saving/saving_lib.py:802: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 14 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


#11. Evaluate on Test Set

In [13]:
test_loss, test_accuracy = best_model.evaluate(X_test, y_test)
print("\nBest Model Test Accuracy:", test_accuracy)

13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.9428 - loss: 0.1432  

Best Model Test Accuracy: 0.9449999928474426


93.25% accuracy on completely unseen test data

#12. Show Best Hyperparameters

In [14]:
best_hp = tuner.get_best_hyperparameters(1)[0]

print("\nBest Hyperparameters:")
print("Best units_1:", best_hp.get('units_1'))
print("Best units_2:", best_hp.get('units_2'))
print("Best dropout:", best_hp.get('dropout_rate'))
print("Best learning rate:", best_hp.get('learning_rate'))


Best Hyperparameters:
Best units_1: 32
Best units_2: 16
Best dropout: 0.2
Best learning rate: 0.001


**INSIGHT**
* first hidden layer - 8 neurons(Initial compression)
* second hidden layer - 24 neurons(Slightly expanded representation)
*30% of neurons randomly turned off during training
* learning rate = 0.01 implies smooth convergence(balanced)


#13. Classification Report

In [15]:
y_pred = np.argmax(best_model.predict(X_test), axis=1)
y_true = np.argmax(y_test, axis=1)

print("\nClassification Report:\n")
print(classification_report(y_true, y_pred))

13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step

Classification Report:

              precision    recall  f1-score   support

           0       0.95      0.97      0.96       105
           1       0.94      0.92      0.93        91
           2       0.96      0.88      0.92        92
           3       0.93      0.99      0.96       112

    accuracy                           0.94       400
   macro avg       0.95      0.94      0.94       400
weighted avg       0.95      0.94      0.94       400



**INSIGHT**
* Class 0: high precision & recall, identifies low-price phones very well
* Class 1: 95% recall
* Class 2: low precision & low recall,harder to distinguish
* Class 3: high precision, slightly lower recall

 macro avg = weighted avg = 0.93 means classes are balanced, model performance is consistent & no class bias.

**CONCLUSION**

An ANN classifier was developed to predict mobile phone price ranges based on device features.
Hyperparameter tuning was performed using Keras Tuner, optimizing the number of neurons, dropout rate, and learning rate.

The best model achieved:

Validation Accuracy: 93.75%

Test Accuracy: 93.25%

Weighted F1-Score: 0.93

The classification report indicates balanced performance across all four price categories, with minimal overfitting.
The model generalizes well to unseen data and successfully captures the relationship between mobile features and price range.